In [1]:
import os

In [2]:
pwd

'/Users/vatsal/Machine Learning/MLOPS/Multilingual_Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
pwd

'/Users/vatsal/Machine Learning/MLOPS/Multilingual_Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [6]:
from src.MultilingualSummarizer.constants import *
from src.MultilingualSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config



In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

/Users/vatsal/Machine Learning/MLOPS/Multilingual_Summarizer/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os, gc, torch
from datasets import load_from_disk
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq, TrainingArguments, Trainer
)

class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train(self):
        # 1) Proper device selection (MPS for Apple GPU)
        if torch.backends.mps.is_available():
            device = "mps"
        elif torch.cuda.is_available():
            device = "cuda"
        else:
            device = "cpu"

        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_mt5 = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_mt5)

        #loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

         # 3) Memory savers for seq2seq on MPS
        model_mt5.gradient_checkpointing_enable()
        if hasattr(model_mt5.config, "use_cache"):
            model_mt5.config.use_cache = False  # must be False with grad checkpointing


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=8,     # effective batch = 8
            eval_strategy="steps",       # <-- correct arg name
            eval_steps=500,
            save_steps=1000000,                # int, not 1e6 float
            logging_steps=10,
            warmup_steps=200,                  # lower warmup to cut steps/memory
            weight_decay=0.01,
            optim="adafactor",                 # big memory win vs AdamW
            dataloader_pin_memory=False,       # CUDA-specific; off on MPS
            report_to="none",
            load_best_model_at_end=False,      # can set True if you also set metric_for_best_model
        )
        trainer = Trainer(model=model_mt5, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        

        
        trainer.train()

        ## Save model
        model_mt5.save_pretrained(os.path.join(self.config.root_dir,"mT5_multilingual-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [11]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.55.4
Uninstalling transformers-4.55.4:
  Successfully uninstalled transformers-4.55.4
Found existing installation: accelerate 1.10.0
Uninstalling accelerate-1.10.0:
  Successfully uninstalled accelerate-1.10.0
  Using cached transformers-4.55.4-py3-none-any.whl.metadata (41 kB)
  Using cached accelerate-1.10.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.55.4-py3-none-any.whl (11.3 MB)
Using cached accelerate-1.10.0-py3-none-any.whl (374 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [15]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-08-24 14:18:43,183: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-24 14:18:43,185: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-24 14:18:43,185: INFO: common: created directory at: artifacts]
[2025-08-24 14:18:43,186: INFO: common: created directory at: artifacts/model_trainer]


RuntimeError: MPS backend out of memory (MPS allocated: 7.14 GiB, other allocations: 12.88 GiB, max allowed: 20.40 GiB). Tried to allocate 732.75 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).